# 05-xrefs
Get xrefs from a variety of sources
- Drugs: 
UMLS has mesh xrefs. From mesh, we can get UNII and CAS. From UNII_FDA, we can get inchikeys 
(lookup using cas or unii). From chembl, we can get chembl IDs from the inchikeys
So: UMLS -> mesh -> unii/cas -> inchikey -> chembl
insane, I know.
- Anatomy: uberon has umls xrefs
- disease: DO has umls, umls has NCI, ICD10PCS, SNOMEDCT_US, ICD10CM, OMIM
- proteins: umls has uniprot xrefs
- biological_process_or_activity/activity_and_behavior: umls has GO
- gene: umls has HGNC and OMIM

In [1]:
import sys
import os
import pickle
%matplotlib inline
import pandas as pd
from numpy import nan
import seaborn as sns
import shelve
import re
from collections import defaultdict, Counter
from tqdm import tqdm
from tqdm.notebook import tqdm as tqdm_notebook
from itertools import chain
from more_itertools import chunked
from collections import Counter
from pprint import pprint
import requests
from pyquery import PyQuery as pq
from wikidataintegrator import wdi_helpers, wdi_core, wdi_login

In [2]:
DATA  = 'data/'

UMLS  = DATA+"2020AA-full/2020AA/META/"
MRCONSO_ENG_ARCHIVE = UMLS+"MRCONSO_ENG.RRF.gz"
MRSAT_ARCHIVE = UMLS+"MRSAT.RRF.gz"

EDGES_BIOLINK_CSV = DATA+"edges_biolink.csv"
NODES_BIOLINK_CSV = DATA+"nodes_biolink.csv"

UNII_RECORDS = DATA+"UNII_Records_23May2020.txt"

XREFS_SHELVE = DATA+"xrefs.shelve"

UBERON_CSV = DATA+"uberon.csv"
DOID_CSV = DATA+"doid.csv"

MESH_XREFS_CSV = DATA+"mesh_xrefs.csv"
NODES_XREF_CSV = DATA+"nodes_xref.csv"

In [3]:
uri_to_curie = lambda s: s.split("/")[-1].replace("_", ":")

In [4]:
nodes = pd.read_csv(NODES_BIOLINK_CSV, index_col=0)

In [5]:
nodes.head()

,LABEL,umls_type,umls_type_label,blm_category
ID,,,,
C1419689,RPL9 gene,T028,Gene or Genome,Gene
C0089147,butyl alcohol,T109,Organic Chemical,ChemicalSubstance
C0661927,peptitergent PD1,T121|T116,"Pharmacologic Substance|Amino Acid, Peptide, o...",Protein
C0380168,glutathione S-conjugate export pump,T123|T116,"Amino Acid, Peptide, or Protein|Biologically A...",Protein
C0141404,SAGM solution,T104|T122,Biomedical or Dental Material|Chemical Viewed ...,ChemicalSubstance


In [6]:
nodes.blm_category.value_counts()

ChemicalSubstance              52455
Protein                        25945
DiseaseOrPhenotypicFeature     24498
Gene                           23116
BiologicalProcessOrActivity     7052
GrossAnatomicalStructure        6885
CellComponent                   1582
AnatomicalEntity                1210
Cell                            1114
ActivityAndBehavior              809
PhenotypicFeature                223
GenomicEntity                    169
Name: blm_category, dtype: int64

## parse UMLS flat file to get all UMLS xrefs
# see: https://www.ncbi.nlm.nih.gov/books/NBK9685/

In [7]:
names = "CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,X".split(",")
umls = pd.read_csv(MRCONSO_ENG_ARCHIVE, delimiter="|", names=names, index_col=None)
# only get CUIs in our list of nodes
umls = umls[umls.CUI.isin(nodes.index)]

/Users/richard/opt/anaconda3/lib/python3.7/site-packages/IPython/core/interactiveshell.py:3063: DtypeWarning: Columns (9,10,13) have mixed types.Specify dtype option on import or set low_memory=False.
  interactivity=interactivity, compiler=compiler, result=result)


In [8]:
umls['xref'] = umls.SAB + ":" + umls.CODE.map(str)
# easy fix to HGNC prefix duplication between SAB and CODE
umls.xref = umls.xref.str.replace("HGNC:HGNC:", "HGNC:")
# fix this MSH MESH nonsense
umls.xref = umls.xref.str.replace("MSH:", "MESH:")
# NCI_FDA is UNII
umls.xref = umls.xref.str.replace("NCI_FDA:", "UNII:")

In [9]:
umls.head(20)

,CUI,LAT,TS,LUI,STT,SUI,ISPREF,AUI,SAUI,SCUI,SDUI,SAB,TTY,CODE,STR,SRL,SUPPRESS,CVF,X,xref
2,C0000039,ENG,P,L0000039,PF,S17175117,N,A28315139,9194921.0,1926948,NaN,RXNORM,IN,1926948,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN,RXNORM:1926948
3,C0000039,ENG,P,L0000039,PF,S17175117,Y,A28572604,NaN,NaN,NaN,MTH,PN,NOCODE,"1,2-dipalmitoylphosphatidylcholine",0,N,256.0,NaN,MTH:NOCODE
4,C0000039,ENG,P,L0000039,VC,S0007564,Y,A0016515,NaN,M0023172,D015060,MSH,MH,D015060,"1,2-Dipalmitoylphosphatidylcholine",0,N,NaN,NaN,MESH:D015060
5,C0000039,ENG,P,L0000039,VC,S1357296,Y,A1317708,NaN,M0023172,D015060,MSH,PM,D015060,"1,2 Dipalmitoylphosphatidylcholine",0,N,NaN,NaN,MESH:D015060
6,C0000039,ENG,S,L0000035,PF,S0007560,Y,A26674543,NaN,M0023172,D015060,MSH,ET,D015060,"1,2-Dihexadecyl-sn-Glycerophosphocholine",0,N,NaN,NaN,MESH:D015060
7,C0000039,ENG,S,L0000035,VO,S1357276,Y,A1317687,NaN,M0023172,D015060,MSH,PM,D015060,"1,2 Dihexadecyl sn Glycerophosphocholine",0,N,NaN,NaN,MESH:D015060
8,C0000039,ENG,S,L0000038,PF,S0007563,Y,A26661070,NaN,M0023172,D015060,MSH,ET,D015060,"1,2-Dipalmitoyl-Glycerophosphocholine",0,N,NaN,NaN,MESH:D015060
9,C0000039,ENG,S,L0000038,VO,S1357295,Y,A1317707,NaN,M0023172,D015060,MSH,PM,D015060,"1,2 Dipalmitoyl Glycerophosphocholine",0,N,NaN,NaN,MESH:D015060
10,C0000039,ENG,S,L0012507,PF,S0033298,N,A18399186,NaN,LP15542-1,NaN,LNC,LPN,LP15542-1,Dipalmitoylphosphatidylcholine,0,N,256.0,NaN,LNC:LP15542-1
11,C0000039,ENG,S,L0012507,PF,S0033298,N,A23513030,NaN,MTHU010538,NaN,LNC,CN,MTHU010538,Dipalmitoylphosphatidylcholine,0,N,256.0,NaN,LNC:MTHU010538


In [10]:
XREF = dict(umls.groupby("CUI")['xref'].apply(set))
XREF = defaultdict(set, XREF)
print(XREF['C0000039'])

{'MTH:NOCODE', 'MESH:D015060', 'LNC:LP15542-1', 'LNC:MTHU010538', 'RXNORM:1926948'}


### Chemicals and drugs

In [11]:
# what xrefs are on chemicals?
chem_umls = nodes[nodes.blm_category == "ChemicalSubstance"].index
xref_chem = {k:v for k,v in XREF.items() if k in chem_umls}
print(len(chem_umls))
c = Counter(list(chain(*[list(map(lambda x:x.split(":",1)[0], y)) for y in xref_chem.values()])))
pprint(c.most_common(25))
# nearly all have a mesh ID. not much of anything else
# neither mesh nor umls have inchikeys, or inchi, or smiles or anything useful for linking out
# blech

52455
[('MESH', 47852),
 ('NCI', 10607),
 ('CHV', 8077),
 ('RXNORM', 7918),
 ('MTH', 7010),
 ('UNII', 6339),
 ('LNC', 5409),
 ('MTHSPL', 4467),
 ('VANDF', 3754),
 ('DRUGBANK', 3437),
 ('ATC', 3150),
 ('CSP', 2876),
 ('PDQ', 2849),
 ('AOD', 1926),
 ('LCH_NW', 1819),
 ('USP', 1782),
 ('NCI_NCI-GLOSS', 1490),
 ('LCH', 1172),
 ('NCI_CTRP', 983),
 ('USPMG', 914),
 ('NCI_DTP', 811),
 ('NCI_DCP', 729),
 ('NCI_CRCH', 234),
 ('HL7V3.0', 166),
 ('SPN', 133)]


In [12]:
pd.set_option("display.width", 120)

URL = "http://id.nlm.nih.gov/mesh/sparql"
PREFIX = """
PREFIX rdf: <http://www.w3.org/1999/02/22-rdf-syntax-ns#>
PREFIX rdfs: <http://www.w3.org/2000/01/rdf-schema#>
PREFIX xsd: <http://www.w3.org/2001/XMLSchema#>
PREFIX owl: <http://www.w3.org/2002/07/owl#>
PREFIX meshv: <http://id.nlm.nih.gov/mesh/vocab#>
PREFIX mesh: <http://id.nlm.nih.gov/mesh/>
"""

def sparql_query(query):
    params = {'query': PREFIX + query, 'format': 'JSON', 'limit': 1000, 'offset': 0}
    r = requests.get(URL, params=params)
    res = [{k: v['value'] for k, v in x.items()} for x in r.json()['results']['bindings']]
    t = tqdm()
    while True:
        t.update(1)
        params['offset'] += 1000
        r = requests.get(URL, params=params).json()['results']['bindings']
        if not r:
            break
        res.extend([{k: v['value'] for k, v in x.items()} for x in r])
    df = pd.DataFrame(res)
    return df

In [13]:
query = """
SELECT distinct ?mesh ?meshLabel ?r ?rr
FROM <http://id.nlm.nih.gov/mesh> WHERE {
  ?mesh meshv:active 1 .
  ?mesh meshv:preferredMappedTo ?p .
  ?p meshv:treeNumber ?treeNum .
  FILTER(STRSTARTS(STR(?treeNum), "http://id.nlm.nih.gov/mesh/D")) .
  ?mesh rdfs:label ?meshLabel .
  ?mesh meshv:preferredConcept [meshv:registryNumber ?r] .
  #OPTIONAL {?mesh meshv:preferredConcept [meshv:relatedRegistryNumber ?rr]}
}
"""
df = sparql_query(query)

218it [06:17,  2.09s/it]

In [14]:
df.r = df.r.replace("0", nan)
df.dropna(subset=["r"], inplace=True)
df = df[~df.r.str.startswith("EC ")]
df.mesh = df.mesh.str.replace("http://id.nlm.nih.gov/mesh/", "")
df.set_index("mesh", inplace=True)

In [15]:
df.to_csv(MESH_XREFS_CSV)
df.head()

,meshLabel,r
mesh,,
C000603935,aripiprazole lauroxil,B786J7A343
C487637,telavancin,XK134822Z0
C070576,"PML protein, human",143220-95-5
C100708,oritavancin,PUG62FRZ2E
C000603533,3-methyl-3-buten-1-ol,KJ25C8CPFA


In [16]:
mesh_xrefs = pd.read_csv(MESH_XREFS_CSV, index_col=0)
mesh_xrefs.r = mesh_xrefs.r.apply(lambda x: "CAS:" + x if "-" in x else "UNII:" + x)
mesh_xrefs = mesh_xrefs.groupby("mesh").r.apply(set).to_dict()
mesh_xrefs = {"MESH:"+k:v for k,v in mesh_xrefs.items()}
len(mesh_xrefs)

44021

In [17]:
for k,v in xref_chem.items():
    for vv in list(v):
        if vv in mesh_xrefs:
            v.update(mesh_xrefs[vv])

In [18]:
# download: 'http://fdasis.nlm.nih.gov/srs/download/srs/UNII_Data.zip'
unii_df = pd.read_csv(UNII_RECORDS, dtype=str, sep='\t', low_memory=False)
unii_df.dropna(subset=['INCHIKEY'], inplace=True)

In [19]:
unii_df.head()

,UNII,PT,RN,EC,NCIT,RXCUI,PUBCHEM,ITIS,NCBI,PLANTS,GRIN,MPNS,INN_ID,MF,INCHIKEY,SMILES,INGREDIENT_TYPE
0,00174624E2,CHF-6333 CATION,1613620-10-2,NaN,NaN,NaN,76285164,NaN,NaN,NaN,NaN,NaN,NaN,C27H28F3N6O3,IHTRPSMRGYWUIM-HSZRJFAPSA-O,COC(=O)C1=C(C)N(C2=NNC(=O)N2[C@@H]1C3=CC=C(C=C...,IONIC MOIETY
1,0129526470,"5,8-DIMETHOXY(1,2,4)TRIAZOLO(1,5-C)PYRIMIDIN-2...",219715-62-5,NaN,NaN,NaN,11446888,NaN,NaN,NaN,NaN,NaN,NaN,C7H9N5O2,DBJPBHJHAPAUQU-UHFFFAOYSA-N,COC1=CN=C(OC)N2N=C(N)N=C12,INGREDIENT SUBSTANCE
3,0377415922,"N-DESMETHYLVENLAFAXINE, (S)-",392332-59-1,NaN,NaN,NaN,9860056,NaN,NaN,NaN,NaN,NaN,NaN,C16H25NO2,MKAFOJAJJMUXLW-OAHLLOKOSA-N,CNC[C@H](C1=CC=C(OC)C=C1)C2(O)CCCCC2,INGREDIENT SUBSTANCE
5,0480546720,HOMOCYCLOLEUCINE HYDROCHLORIDE,39692-17-6,254-594-3,NaN,NaN,2724466,NaN,NaN,NaN,NaN,NaN,NaN,C7H13NO2.ClH,GTKXSYHXQSKWNP-UHFFFAOYSA-N,Cl.NC1(CCCCC1)C(O)=O,INGREDIENT SUBSTANCE
6,0503177591,SUCCINALDEHYDE,638-37-9,211-333-8,NaN,NaN,12524,NaN,NaN,NaN,NaN,NaN,NaN,C4H6O2,PCSMJKASWLYICJ-UHFFFAOYSA-N,O=CCCC=O,INGREDIENT SUBSTANCE


In [20]:
n=0
for k,v in tqdm_notebook(xref_chem.items()):
    for vv in list(v):
        if vv.startswith("UNII:"):
            xref = vv.replace("UNII:", "")
            s = unii_df.query("UNII == @xref").INCHIKEY
            if not s.empty:
                n+=1
                v.add("INCHIKEY:" + list(s)[0])

In [21]:
xref_inchi = {k:v for k,v in xref_chem.items() if any(vv.startswith("INCHIKEY:") for vv in v)}
xref_inchi = {k:[vv for vv in v if vv.startswith("INCHIKEY:")][0].replace("INCHIKEY:", "") for k,v in xref_inchi.items()}
print(len(xref_inchi))
list(xref_inchi.items())[:4]

10324


[('C0000248', 'CZIHNRWJTSTCEX-UHFFFAOYSA-N'),
 ('C0000294', 'XOGTZOOQQBDUSI-UHFFFAOYSA-M'),
 ('C0000378', 'QXWYKJLNLSIPIN-JGVFFNPUSA-N'),
 ('C0000399', 'CBOKZNLSFMZJJA-PEBGCTIMSA-N')]

In [22]:
url = "https://www.ebi.ac.uk/chembl/api/data/molecule?molecule_structures__standard_inchi_key__in={}&format=json&limit=100"
for chunk in tqdm(chunked(xref_inchi.items(), 100), total=len(xref_inchi)/100):
    chunk = dict(chunk)
    chunk = {v:k for k,v in chunk.items()}
    inchis = ",".join(chunk)
    mols = requests.get(url.format(inchis)).json()['molecules']
    for m in mols:
        chembl = m['molecule_chembl_id']
        inchi = m['molecule_structures']['standard_inchi_key']
        XREF[chunk[inchi]].add("CHEMBL:" + chembl)


  0%|          | 0/103.24 [00:00<?, ?it/s]
  1%|          | 1/103.24 [00:03<06:32,  3.84s/it]
  2%|▏         | 2/103.24 [00:05<05:30,  3.26s/it]
  3%|▎         | 3/103.24 [00:07<04:49,  2.89s/it]
  4%|▍         | 4/103.24 [00:09<04:16,  2.59s/it]
  5%|▍         | 5/103.24 [00:13<04:36,  2.82s/it]
  6%|▌         | 6/103.24 [00:20<06:48,  4.20s/it]
  7%|▋         | 7/103.24 [00:25<07:19,  4.57s/it]
  8%|▊         | 8/103.24 [00:33<08:36,  5.42s/it]
  9%|▊         | 9/103.24 [00:38<08:32,  5.43s/it]
 10%|▉         | 10/103.24 [00:45<09:13,  5.93s/it]
 11%|█         | 11/103.24 [00:50<08:30,  5.54s/it]
 12%|█▏        | 12/103.24 [00:57<09:03,  5.95s/it]
 13%|█▎        | 13/103.24 [01:04<09:28,  6.30s/it]
 14%|█▎        | 14/103.24 [01:08<08:33,  5.76s/it]
 15%|█▍        | 15/103.24 [01:14<08:10,  5.55s/it]
 15%|█▌        | 16/103.24 [01:19<07:49,  5.38s/it]
 16%|█▋        | 17/103.24 [01:22<06:57,  4.84s/it]
 17%|█▋        | 18/103.24 [01:27<06:59,  4.92s/it]
 18%|█▊        | 19/103.24 [0

In [23]:
len({k:v for k,v in XREF.items() if any(vv.startswith("CHEMBL:") for vv in v)})

8043

In [24]:
with open(XREFS_SHELVE, 'wb') as f:
    pickle.dump(XREF, f)

## UBERON, uses uberon.csv generated from the uberon.owl file previously downloaded (see README)

In [25]:
df = pd.read_csv(UBERON_CSV)
df = df[df.xref.str.startswith("UMLS:")]
df.xref = df.xref.str.replace("UMLS:", "")
df.item = df.item.apply(uri_to_curie)
df.head()

,item,xref
9755,UBERON:0007227,C0228757
11788,UBERON:0002965,C0175263
11853,UBERON:0001532,C0007276
11897,UBERON:0014636,C0228627
11904,UBERON:0002610,C0152411


In [26]:
s = df.groupby("xref")['item'].apply(set)
for umls, x in dict(s).items():
    XREF[umls].update(x)

In [27]:
XREF['C1272528']

{'MTH:NOCODE', 'UBERON:0006472'}

## DOID, uses doid.csv generated from the doi.owl file previously downloaded (see README)

In [28]:
df = pd.read_csv(DOID_CSV)
df.dropna(inplace=True)
df = df[df.xref.str.startswith("UMLS_CUI:")]
df.xref = df.xref.str.replace("UMLS_CUI:", "")
df.item = df.item.apply(uri_to_curie)
df.head()

,item,xref
6,DOID:3864,C1332188
7,DOID:3864,C0278876
21,DOID:699,C0162670
27,DOID:0111831,C1844936
34,DOID:0111618,C1853116


In [29]:
s = df.groupby("xref")['item'].apply(set)
for umls, x in dict(s).items():
    XREF[umls].update(x)

In [30]:
XREF['C0263518']

{'DOID:1943',
 'HPO:HP:0025470',
 'ICD9CM:704.02',
 'MTH:NOCODE',
 'NCI:C112200',
 'NCI_NICHD:C112200'}

In [31]:
XREF['C0591520']

{'CHV:0000041179', 'MESH:D000068298'}

## proteins

In [32]:
# See README regarding MRSAT_ARCHIVE
names = list("abcdefghijklmn")
iter_csv = pd.read_csv(MRSAT_ARCHIVE, delimiter="|", names=names, index_col=None, chunksize=1000000)
chunks = []
umls_uniprot = dict()
for chunk in tqdm(iter_csv, total=67668372/1000000):
    chunk.fillna(method='ffill', inplace=True)
    chunk = chunk[chunk.i == "SWISS_PROT"]
    d = dict(zip(chunk.a, chunk.k))
    umls_uniprot.update(d)


  0%|          | 0/67.668372 [00:00<?, ?it/s]
  1%|▏         | 1/67.668372 [00:03<03:29,  3.14s/it]
  3%|▎         | 2/67.668372 [00:06<03:21,  3.07s/it]
  4%|▍         | 3/67.668372 [00:08<03:12,  2.98s/it]
  6%|▌         | 4/67.668372 [00:11<03:03,  2.88s/it]
  7%|▋         | 5/67.668372 [00:14<02:57,  2.84s/it]
  9%|▉         | 6/67.668372 [00:16<02:53,  2.81s/it]
 10%|█         | 7/67.668372 [00:19<02:49,  2.80s/it]
 12%|█▏        | 8/67.668372 [00:22<02:44,  2.76s/it]
 13%|█▎        | 9/67.668372 [00:25<02:41,  2.75s/it]
 15%|█▍        | 10/67.668372 [00:27<02:40,  2.78s/it]
 16%|█▋        | 11/67.668372 [00:30<02:39,  2.81s/it]
 18%|█▊        | 12/67.668372 [00:33<02:32,  2.74s/it]
 19%|█▉        | 13/67.668372 [00:36<02:29,  2.74s/it]
 21%|██        | 14/67.668372 [00:38<02:24,  2.69s/it]
 22%|██▏       | 15/67.668372 [00:41<02:20,  2.67s/it]
 24%|██▎       | 16/67.668372 [00:44<02:22,  2.76s/it]
 25%|██▌       | 17/67.668372 [00:47<02:21,  2.79s/it]/Users/richard/opt/anaconda3

In [33]:
len(umls_uniprot)

4916

In [34]:
for umls, uniprot in umls_uniprot.items():
    XREF[umls].add("UNIPROT:" + uniprot)

In [35]:
XREF['C0215993']

{'MESH:C081092',
 'MTH:NOCODE',
 'NCI:C127008',
 'NCI_CTRP:C127008',
 'UNIPROT:Q04756'}

In [36]:
with open(XREFS_SHELVE, 'wb') as f:
    pickle.dump(XREF, f)

In [37]:
nodes['xrefs'] = nodes.index.map(lambda x: ";".join(XREF.get(x,list())))

In [39]:
nodes.head(20)

,LABEL,umls_type,umls_type_label,blm_category,xrefs
ID,,,,,
C1419689,RPL9 gene,T028,Gene or Genome,Gene,MTH:NOCODE;HGNC:10369;OMIM:603686
C0089147,butyl alcohol,T109,Organic Chemical,ChemicalSubstance,MTHSPL:8PJ61P6TS3;MTH:NOCODE;VANDF:4025696;MES...
C0661927,peptitergent PD1,T121|T116,"Pharmacologic Substance|Amino Acid, Peptide, o...",Protein,MESH:C083798
C0380168,glutathione S-conjugate export pump,T123|T116,"Amino Acid, Peptide, or Protein|Biologically A...",Protein,MTH:NOCODE;MESH:C086860
C0141404,SAGM solution,T104|T122,Biomedical or Dental Material|Chemical Viewed ...,ChemicalSubstance,MESH:C037839
C0152814,epididymal tuberculosis,T047,Disease or Syndrome,DiseaseOrPhenotypicFeature,DOID:1251;ICD9CM:016.4;CHV:0000017302
C1366509,DLC1 gene,T028,Gene or Genome,Gene,MTH:NOCODE;NCI_NCI-HGNC:2897;HGNC:2897;OMIM:60...
C0380056,anti-Mullerian hormone type II receptor,T192|T116,"Amino Acid, Peptide, or Protein|Receptor",Protein,MTH:NOCODE;MESH:C084621
C0379924,ZD 8731,T109,Organic Chemical,ChemicalSubstance,MESH:C081112;CAS:143494-72-8


In [40]:
nodes.to_csv(NODES_XREF_CSV)